In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

if "Pokemon_ds" not in locals():
    #requesting all Pokemon with names, stats, typing, abilities, and movesets
    ds1_Pokemon = requests.get("https://pastebin.com/raw/0DV5kDEy").text
    ds2_Pokemon = requests.get("https://pastebin.com/raw/8PGBSP39").text
    Pokemon_ds  = ds1_Pokemon+ds2_Pokemon
    Pokemon_ds  = Pokemon_ds.split("\r\n\r\n")[1:len(Pokemon_ds)-1]

    #setup dataframe
    data=pd.DataFrame(columns=["p_name","HP","Att","Def","SAtt","SDef","Spd","tStats",
                             "type1","type2","Ab1","Ab2","HAb","moveset"])

    for i,mon in enumerate(Pokemon_ds):
        
        if mon.split(" ")[2] == "-":
            name  = " ".join(mon.split(" ")[0:2])
        elif mon.split(" ")[0].replace("\r\n","").strip() == "Flab颩":
            name  = "Flabébé"
        else:
            name  = mon.split(" ")[0].replace("\r\n","").strip()
        stats     = [int(stat) for stat in mon.split(" -")[1].split("(")[0].strip().split("/")] 
        tStats    = int(mon.split(" -")[1].strip().split(" ")[2].replace(")",""))
        types     = mon.split(" -")[2].strip().split("/")
        abilities = mon.split("- ")[3].split("\n")[0].split("/")
            
        try:
            lvl_moves = [mv.strip().split("@")[0].strip() for i, mv in enumerate(mon[mon.index("Learned Moves:"):mon.index("TM Moves:")].replace("Learned Moves:\r\n","").split("\r\n")) if "Egg Moves:" not in mv ]
        except:
            lvl_moves = [mv.strip().split("@")[0].strip() for i, mv in enumerate(mon[mon.index("Learned Moves:"):len(mon)].replace("Learned Moves:\r\n","").split("\r\n"))]
        try:
            egg_moves = [mv.strip() for mv in mon[mon.index("Egg Moves:"):mon.index("TM Moves:")].replace("Egg Moves:\r\n","").strip().split(",")]
        except:
            egg_moves = []
            
        try:
            TM_moves  = [mv.strip() for mv in mon[mon.index("TM Moves:"):len(mon)].replace("TM Moves:\r\n","").strip().split(",")]
        except:
            TM_moves  = []
        moves = [*lvl_moves,*egg_moves,*TM_moves]
        
        data.loc[len(data)] = [name,stats[0],stats[1],stats[2],stats[3],stats[4],stats[5],
                               tStats,types[0],types[1],abilities[0],abilities[1],abilities[2],moves]